# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 16 2023 12:05PM,259452,10,PRF-55579,Bio-PRF,Released
1,Mar 16 2023 12:05PM,259452,10,PRF-55644,Bio-PRF,Released
2,Mar 16 2023 12:05PM,259452,10,PRF-55650,Bio-PRF,Released
3,Mar 16 2023 12:05PM,259452,10,PRF-55655,Bio-PRF,Released
4,Mar 16 2023 12:05PM,259452,10,PRF-55719,Bio-PRF,Released
5,Mar 16 2023 12:05PM,259452,10,PRF-55724,Bio-PRF,Released
6,Mar 16 2023 12:05PM,259452,10,PRF-55729,Bio-PRF,Released
7,Mar 16 2023 12:05PM,259452,10,PRF-55733,Bio-PRF,Released
8,Mar 16 2023 12:05PM,259452,10,PRF-55747,Bio-PRF,Released
9,Mar 16 2023 12:05PM,259452,10,PRF-55750,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
10,259442,Released,1
11,259443,Released,1
12,259450,Released,6
13,259451,Released,1
14,259452,Released,16


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
259442,NaN,1.0
259443,NaN,1.0
259450,NaN,6.0
259451,NaN,1.0
259452,NaN,16.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
259385,2.0,4.0
259414,37.0,17.0
259416,0.0,6.0
259432,2.0,3.0
259433,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
259385,2,4
259414,37,17
259416,0,6
259432,2,3
259433,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,259385,2,4
1,259414,37,17
2,259416,0,6
3,259432,2,3
4,259433,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,259385,2,4
1,259414,37,17
2,259416,,6
3,259432,2,3
4,259433,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 16 2023 12:05PM,259452,10,Bio-PRF
16,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC"
17,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc."
23,Mar 16 2023 11:12AM,259443,10,Bio-PRF
24,Mar 16 2023 11:08AM,259442,10,Bio-PRF
25,Mar 16 2023 10:35AM,259439,19,"GCH Granules USA, Inc."
31,Mar 16 2023 10:22AM,259436,19,"GCH Granules USA, Inc."
36,Mar 16 2023 10:18AM,259385,15,"Alliance Pharma, Inc."
42,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc."
47,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 16 2023 12:05PM,259452,10,Bio-PRF,,16
1,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",,1
2,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",,6
3,Mar 16 2023 11:12AM,259443,10,Bio-PRF,,1
4,Mar 16 2023 11:08AM,259442,10,Bio-PRF,,1
5,Mar 16 2023 10:35AM,259439,19,"GCH Granules USA, Inc.",,6
6,Mar 16 2023 10:22AM,259436,19,"GCH Granules USA, Inc.",,5
7,Mar 16 2023 10:18AM,259385,15,"Alliance Pharma, Inc.",2,4
8,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",,5
9,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",2,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 12:05PM,259452,10,Bio-PRF,16,
1,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",1,
2,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",6,
3,Mar 16 2023 11:12AM,259443,10,Bio-PRF,1,
4,Mar 16 2023 11:08AM,259442,10,Bio-PRF,1,
5,Mar 16 2023 10:35AM,259439,19,"GCH Granules USA, Inc.",6,
6,Mar 16 2023 10:22AM,259436,19,"GCH Granules USA, Inc.",5,
7,Mar 16 2023 10:18AM,259385,15,"Alliance Pharma, Inc.",4,2
8,Mar 16 2023 10:10AM,259433,19,"GCH Granules USA, Inc.",5,
9,Mar 16 2023 10:01AM,259432,19,"GCH Granules USA, Inc.",3,2


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 12:05PM,259452,10,Bio-PRF,16,
1,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",1,
2,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",6,
3,Mar 16 2023 11:12AM,259443,10,Bio-PRF,1,
4,Mar 16 2023 11:08AM,259442,10,Bio-PRF,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 12:05PM,259452,10,Bio-PRF,16,NaN
1,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",1,NaN
2,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",6,NaN
3,Mar 16 2023 11:12AM,259443,10,Bio-PRF,1,NaN
4,Mar 16 2023 11:08AM,259442,10,Bio-PRF,1,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 16 2023 12:05PM,259452,10,Bio-PRF,16,0.0
1,Mar 16 2023 11:56AM,259451,19,"NAPP Technologies, LLC",1,0.0
2,Mar 16 2023 11:49AM,259450,19,"GCH Granules USA, Inc.",6,0.0
3,Mar 16 2023 11:12AM,259443,10,Bio-PRF,1,0.0
4,Mar 16 2023 11:08AM,259442,10,Bio-PRF,1,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,778337,18,0.0
15,259385,4,2.0
19,2075471,49,39.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,778337,18,0.0
1,15,259385,4,2.0
2,19,2075471,49,39.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,18,0.0
1,15,4,2.0
2,19,49,39.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,15,Released,4.0
2,19,Released,49.0
3,10,Executing,0.0
4,15,Executing,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19
Status,,,
Executing,0.0,2.0,39.0
Released,18.0,4.0,49.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19
0,Executing,0.0,2.0,39.0
1,Released,18.0,4.0,49.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19
0,Executing,0.0,2.0,39.0
1,Released,18.0,4.0,49.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()